🌍 Unified CO₂ Monitoring & RAG Assistant - Notebook Template

🧩 1. Install Required Packages

In [1]:
!pip install pandas sqlalchemy pymysql gradio plotly requests

⚙️ 2. Import Required Libraries

In [2]:
import pandas as pd
import requests
from sqlalchemy import create_engine, text
import gradio as gr
import plotly.express as px
import pymysql as pymysql

🔐 3. API & Configuration Settings

In [3]:
# ---------- CONFIGURATION ----------
TAVILY_API_KEY = "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"
WATSONX_API_KEY = "eyJraWQiOiIyMDI1MDUzMDA4NDYiLCJhbGciOiJSUzI1NiJ9.eyJpYW1faWQiOiJJQk1pZC02OTQwMDBKMEZGIiwiaWQiOiJJQk1pZC02OTQwMDBKMEZGIiwicmVhbG1pZCI6IklCTWlkIiwic2Vzc2lvbl9pZCI6IkMtZmI3OTgxYTMtMTFjZS00YTRkLWIyODktY2RmYmUxYWYxOWZhIiwic2Vzc2lvbl9leHBfbWF4IjoxNzQ4NzczOTQ5LCJzZXNzaW9uX2V4cF9uZXh0IjoxNzQ4Njk0Nzc5LCJqdGkiOiI4ZTk5OTYzYi1jMDA0LTQ3NmQtOGFlMy1hZTc1NDkyZDNlNDgiLCJpZGVudGlmaWVyIjoiNjk0MDAwSjBGRiIsImdpdmVuX25hbWUiOiJCYWxhZ2FqYXJhaiIsImZhbWlseV9uYW1lIjoiUHJhYmhha2FyIiwibmFtZSI6IkJhbGFnYWphcmFqIFByYWJoYWthciIsImVtYWlsIjoiYmFsYXByYWJoYTYwQGdtYWlsLmNvbSIsInN1YiI6ImJhbGFwcmFiaGE2MEBnbWFpbC5jb20iLCJhdXRobiI6eyJzdWIiOiJiYWxhcHJhYmhhNjBAZ21haWwuY29tIiwiaWFtX2lkIjoiSUJNaWQtNjk0MDAwSjBGRiIsIm5hbWUiOiJCYWxhZ2FqYXJhaiBQcmFiaGFrYXIiLCJnaXZlbl9uYW1lIjoiQmFsYWdhamFyYWoiLCJmYW1pbHlfbmFtZSI6IlByYWJoYWthciIsImVtYWlsIjoiYmFsYXByYWJoYTYwQGdtYWlsLmNvbSJ9LCJhY2NvdW50Ijp7InZhbGlkIjp0cnVlLCJic3MiOiIzYTk0MWU2MjgyNjA0ZTc5YWJkMTA4NjE1Mzc2NWEzMCIsImltc191c2VyX2lkIjoiMTM3MTIzMjkiLCJpbXMiOiIyOTk2MzQyIn0sImlhdCI6MTc0ODY4NzU3NiwiZXhwIjoxNzQ4Njg4Nzc2LCJpc3MiOiJodHRwczovL2lhbS5jbG91ZC5pYm0uY29tL2lkZW50aXR5IiwiZ3JhbnRfdHlwZSI6InVybjppYm06cGFyYW1zOm9hdXRoOmdyYW50LXR5cGU6cGFzc2NvZGUiLCJzY29wZSI6ImlibSBvcGVuaWQiLCJjbGllbnRfaWQiOiJieCIsImFjciI6MSwiYW1yIjpbInB3ZCJdfQ.VYxXD5W9KYKjfJkeZ17pf4p2mwyHFEBjNq9l8S3GEF0evCeig9Dkoo5u3XfHcxl4kNpXcsMxHDOxNrPvHFWyEMv7IeCAj2IThu1nAt7FKyU8py2Iaijfaqu-s9SDOeEz-FReAn9yVdDHOhb1uPjCMTc9eDakFrpaIZ1HQDvW3M5EpFytiRTpZAMV0Jb7VFunphcy11IiSFJyVivm8yBuOyRsABT9ZQ2wqqBI3sHGYRwLBwFjhmFJn4fWIV_fq-wZCUikeua0NrPR1P5GhkkV6AwMARIEaqDYC2XM4WVIB_pAI6PGFz5QiOYsndkmX4P7hifmJDhRHLMaIG6GhWFK8w"
PROJECT_ID = "fda1917e-2ed7-4810-a89a-78ed57a005dc"
GRANITE_MODEL_ID = "ibm/granite-3-8b-instruct"
IBM_URL = "https://us-south.ml.cloud.ibm.com/ml/v1/text/generation?version=2023-05-29"
CO2_THRESHOLD = 1000  # ppm
CARBON_CREDIT_PRICE = 15  # USD per credit

DB_CONFIG = {
   "username": "sql12781205",
   "password": "NpRdaBvMws",
   "host": "sql12.freesqldatabase.com",
   "port": "3306",
   "database": "sql12781205"
}

🔌 4. Database Connection

In [4]:
# ---------- DATABASE CONNECTION ----------
try:
    engine = create_engine(
        f"mysql+pymysql://{DB_CONFIG['username']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
    )
except Exception:
    raise Exception("🚫 Could not connect to database. Please check your credentials or internet connection.")

🏢 5. Building Asset Management

In [5]:
# ---------- BUILDING ASSETS FUNCTIONS ----------
def save_building_assets(building_name, has_solar, hvac, insulation, lighting, notes):
    try:
        # Checking for a valid building name input
        if not building_name:
            return "⚠️ Building name cannot be empty."
        
        with engine.begin() as conn:
            # Check if the building already exists
            result = conn.execute(
                text("SELECT COUNT(*) FROM building_assets WHERE building_name = :bname"),
                {"bname": building_name}
            )
            count = result.scalar()  # .scalar() returns the first column of the first row (i.e., the count value)

            if count == 0:
                # First time entry: INSERT
                try:
                    conn.execute(
                        text("""
                            INSERT INTO building_assets 
                            (building_name, has_solar_panels, hvac_type, insulation_quality, lighting_type, notes) 
                            VALUES (:bname, :solar, :hvac, :insul, :light, :notes)
                        """),
                        {
                            "bname": building_name,
                            "solar": has_solar,
                            "hvac": hvac,
                            "insul": insulation,
                            "light": lighting,
                            "notes": notes
                        }
                    )
                    return "✅ New building asset added successfully."
                except Exception as insert_error:
                    print(f"Error inserting building asset: {str(insert_error)}")
                    return "⚠️ Failed to add new building asset. Please check your inputs or try again later."

            else:
                # Building exists: UPDATE
                try:
                    conn.execute(
                        text("""
                            UPDATE building_assets
                            SET 
                                has_solar_panels = :solar,
                                hvac_type = :hvac,
                                insulation_quality = :insul,
                                lighting_type = :light,
                                notes = :notes
                            WHERE building_name = :bname
                        """),
                        {
                            "bname": building_name,
                            "solar": has_solar,
                            "hvac": hvac,
                            "insul": insulation,
                            "light": lighting,
                            "notes": notes
                        }
                    )
                    return "✅ Building asset updated successfully."
                except Exception as update_error:
                    print(f"Error updating building asset: {str(update_error)}")
                    return "⚠️ Failed to update building asset. Please check your inputs or try again later."

    except Exception as general_error:
        # Logging the general error for debugging
        print(f"General error occurred: {str(general_error)}")
        return "⚠️ Sorry, there was a problem saving the building information. Please check your inputs or try again later."


def load_building_assets():
    try:
        df = pd.read_sql("SELECT * FROM building_assets ORDER BY timestamp DESC LIMIT 10", con=engine)
        return df
    except:
        return pd.DataFrame(columns=["Error"])

🔍 6. Web Search (Tavily)

In [6]:
# ---------- WEB SEARCH ----------
def search_web_tavily(query):
    try:
        response = requests.post(
            "https://api.tavily.com/search",
            json={"api_key": TAVILY_API_KEY, "query": query, "max_results": 3},
        )
        if response.status_code == 200:
            results = response.json()
            return "\n".join([f"- {r['content']}" for r in results['results']])
    except Exception:
        pass
    return "🔍 No relevant web results found at this time."

🤖 7. IBM WatsonX API Call

In [7]:
# ---------- IBM WATSONX QUERY ----------
def call_ibm_watsonx(prompt):
    try:
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {WATSONX_API_KEY}"
        }
        body = {
            "input": prompt,
            "parameters": {
                "decoding_method": "greedy",
                "max_new_tokens": 300,
                "repetition_penalty": 1.1,
            },
            "model_id": GRANITE_MODEL_ID,
            "project_id": PROJECT_ID
        }
        response = requests.post(IBM_URL, json=body, headers=headers)
        if response.status_code == 200:
            return response.json()['results'][0]['generated_text'].strip()
        return "⚠️ AI could not generate an answer. Please try again later."
    except:
        return "⚠️ AI service is temporarily unavailable. Please try again."


📊 8. Fetch Real-Time & Yearly Data

In [8]:
# ---------- DATA FETCHING ----------
def fetch_data():
    try:
        df = pd.read_sql("SELECT * FROM realtime_emission_data ORDER BY timestamp DESC", con=engine)
        recent_10 = df.head(70)
        chart = px.line(df.sort_values("timestamp"), x="timestamp", y="co2_sensor_data", title="📉 Real-time CO₂ Levels")
        chart.update_layout(xaxis_title="Timestamp", yaxis_title="CO2 Level (ppm)", height=400)

        year_df = pd.read_sql("SELECT * FROM yearly_emission_summary ORDER BY year DESC LIMIT 2", con=engine)
        if len(year_df) >= 2:
            year_df = year_df.sort_values(by="year")
            year_df["total_emission"] = year_df["total_power"] + year_df["total_gas"]
            carbon_credit_value = max(0, year_df.iloc[0]["total_emission"] - year_df.iloc[1]["total_emission"])
            credit_earned = carbon_credit_value / 1000
            estimated_dollars = credit_earned * CARBON_CREDIT_PRICE

            carbon_credit_summary = pd.DataFrame({
                "Year": [year_df.iloc[1]["year"]],
                "Emission": [year_df.iloc[1]["total_emission"]],
                "Carbon Reduced": [carbon_credit_value],
                "Carbon Credit Earned": [credit_earned],
                "Estimated $ Earned": [estimated_dollars]
            })
        else:
            carbon_credit_summary = pd.DataFrame({
                "Error": ["⚠️ Not enough yearly data for carbon credit calculation."]
            })

        alert = ""
        if df["co2_sensor_data"].iloc[0] > CO2_THRESHOLD:
            alert = f"🚨 High CO₂ Alert: Levels exceeded {CO2_THRESHOLD} ppm!\n"
            suggestion_prompt = f"CO₂ levels have exceeded {CO2_THRESHOLD} ppm. Suggest sustainability actions for a smart building."
            alert += "💡 " + call_ibm_watsonx(suggestion_prompt)

        return recent_10, chart, carbon_credit_summary, alert
    except Exception:
        return pd.DataFrame(), None, pd.DataFrame(), "❌ Couldn't load data. Please try again shortly."

💬 9. Prompt Generator for RAG

In [9]:
# ---------- PROMPT GENERATION ----------
def generate_prompt(realtime, monthly, yearly, carbon_credit, web, user_question, building_assets):
    context = f"""
[REAL-TIME DATA]
{realtime.head(70).to_markdown(index=False)}

[MONTHLY SUMMARY]
{monthly.head(5).to_markdown(index=False)}

[YEARLY ANALYSIS]
{yearly.head(5).to_markdown(index=False)}

[CARBON CREDIT SUMMARY]
{carbon_credit.to_markdown(index=False)}

[BUILDING ASSET INFORMATION]
{building_assets.to_markdown(index=False)}

[WEB INSIGHTS]
{web}
"""
    return f"""
You are an intelligent environmental assistant trained to analyze and explain CO₂ emissions for a smart building system.

You are allowed to answer questions related to:
- Real-time CO₂ levels and recent trends,
- Monthly or yearly emission summaries and comparisons,
- Carbon reduction strategies or suggestions,
- Carbon credits: including how they are calculated, their environmental value, and how they can be monetized,
- Building asset information such as HVAC systems, insulation quality, lighting types, and solar panel installations,
- Any relevant insights found in the provided web search results.

If the user asks something unrelated to these topics, respond with:
"Out of scope: I only answer questions related to CO₂ emissions, carbon credits, building assets, and sustainability topics."

Use the following context to help answer the user's question:

{context}

User Question:
{user_question}

Answer:
"""

🔄 10. Retrieval-Augmented Generation (RAG) Pipeline

In [10]:
# ---------- RAG PIPELINE ----------
def rag_pipeline(user_question):
    try:
        realtime, _, carbon_credit, _ = fetch_data()
        monthly = pd.read_sql("SELECT * FROM monthly_emission_summary ORDER BY month DESC LIMIT 12", con=engine)
        yearly = pd.read_sql("SELECT * FROM yearly_emission_summary ORDER BY year DESC LIMIT 5", con=engine)
        building_assets = load_building_assets()
        web_context = search_web_tavily(user_question)
        prompt = generate_prompt(realtime, monthly, yearly, carbon_credit, web_context, user_question, building_assets)
        answer = call_ibm_watsonx(prompt)
        graph = px.line(realtime.sort_values("timestamp"), x="timestamp", y="co2_sensor_data", title="📊 Real-time CO₂ Graph")
        return realtime, monthly, yearly, carbon_credit, web_context, prompt, answer, graph
    except:
        return pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), "", "", "⚠️ Unable to process your request at the moment.", None

📝 11. Record User Feedback

In [11]:
# ---------- FEEDBACK HANDLER ----------
def record_feedback(question, answer, rating, comment):
    try:
        with engine.begin() as conn:
            conn.execute(
                text("INSERT INTO ai_feedback (question, answer, rating, comment) VALUES (:q, :a, :r, :c)"),
                {"q": question, "a": answer, "r": int(rating), "c": comment}
            )
        return "✅ Thank you! Your feedback has been saved."
    except:
        return "⚠️ Could not save your feedback. Please try again later."

🎛️ 12. Run Gradio App 

In [12]:
# ---------- GRADIO INTERFACE ----------
with gr.Blocks(title="🌍 Unified CO₂ Dashboard + RAG Assistant") as demo:
    gr.Markdown("## 🌿 Real-Time CO₂ Monitoring & Smart Sustainability Assistant")

    with gr.Tab("📈 Live Dashboard"):
        gr.Markdown("### 📊 Real-Time CO₂ Emission Monitoring")
        with gr.Row():
            table_output = gr.Dataframe(label="📄 Latest 10 CO₂ Records")
            chart_output = gr.Plot(label="📈 CO₂ Trend Chart")

        carbon_credit_output = gr.Dataframe(label="💰 Carbon Credit Summary")
        alert_output = gr.Textbox(label="⚠️ Alerts & Sustainability Suggestions", lines=4, interactive=False)
        refresh_button = gr.Button("🔄 Refresh Data")
        refresh_button.click(fn=fetch_data, outputs=[table_output, chart_output, carbon_credit_output, alert_output])
        demo.load(fetch_data, outputs=[table_output, chart_output, carbon_credit_output, alert_output])

    with gr.Tab("💬 Ask CO₂ Assistant"):
        gr.Markdown("### 🤖 Ask me about emissions, trends, or carbon credits")

        with gr.Row():
            question = gr.Textbox(label="📝 Your Question", placeholder="e.g., How much emission was reduced this year?", lines=2)
            ask_btn = gr.Button("🔍 Ask Assistant")

        with gr.Row():
            realtime_df = gr.Dataframe(label="📟 Real-Time Data")
            monthly_df = gr.Dataframe(label="📆 Monthly Summary")

        with gr.Row():
            yearly_df = gr.Dataframe(label="📅 Yearly Summary")
            carbon_credit_df = gr.Dataframe(label="🌱 Carbon Credit Insights")

        with gr.Row():
            web_output = gr.Textbox(label="🌐 Web Insights", lines=6, interactive=False)

        with gr.Accordion("🧠 AI Prompt (for debugging)", open=False):
            prompt_out = gr.Textbox(label="Prompt Sent to WatsonX", lines=12, interactive=False)

        ai_answer = gr.Textbox(label="🧠 Assistant Answer", lines=6, interactive=False)
        plot = gr.Plot(label="📉 Real-Time CO₂ Graph")

        with gr.Row():
            rating = gr.Radio(choices=[1, 2, 3, 4, 5], label="⭐ Rate this Answer")
            comment = gr.Textbox(label="💬 Feedback", placeholder="Let us know how we can improve.", lines=2)
            submit_feedback = gr.Button("Submit Feedback")
            feedback_msg = gr.Textbox(label="✅ Feedback Status", interactive=False)

        ask_btn.click(
            fn=rag_pipeline,
            inputs=[question],
            outputs=[realtime_df, monthly_df, yearly_df, carbon_credit_df, web_output, prompt_out, ai_answer, plot]
        )

        submit_feedback.click(
            fn=record_feedback,
            inputs=[question, ai_answer, rating, comment],
            outputs=[feedback_msg]
        )

    # ---------- NEW BUILDING ASSETS TAB ----------
    with gr.Tab("🏢 Building Assets"):
        gr.Markdown("### 🏗️ Input Your Building's Asset Details")

        with gr.Row():
            building_name = gr.Textbox(label="🏢 Building Name")
            has_solar = gr.Checkbox(label="🔋 Has Solar Panels?")
            hvac_type = gr.Textbox(label="🌬️ HVAC Type (e.g., Central, Split, etc.)")
            insulation_quality = gr.Textbox(label="🧱 Insulation Quality (e.g., High, Medium, Low)")
            lighting_type = gr.Textbox(label="💡 Lighting Type (e.g., LED, CFL, Incandescent)")

        notes = gr.Textbox(label="🗒️ Additional Notes", lines=3)
        save_button = gr.Button("💾 Save Asset Info")
        save_status = gr.Textbox(label="📥 Status", interactive=False)
        asset_data_output = gr.Dataframe(label="📋 Recent Asset Entries")

        save_button.click(
            fn=save_building_assets,
            inputs=[building_name, has_solar, hvac_type, insulation_quality, lighting_type, notes],
            outputs=[save_status]
        )

        demo.load(fn=load_building_assets, outputs=[asset_data_output])

demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7935
* Running on public URL: https://40bb433ffea8be2d45.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
